### Connect to Duckdb and Inspect Database Schema


In [7]:
import duckdb

with duckdb.connect("data/sakila.duckdb") as conn:
    desc = conn.sql("DESC;").df()
desc

,database,schema,name,column_names,column_types,temporary
0,sakila,staging,_dlt_loads,"[load_id, schema_name, status, inserted_at, sc...","[VARCHAR, VARCHAR, BIGINT, TIMESTAMP WITH TIME...",False
1,sakila,staging,_dlt_pipeline_state,"[version, engine_version, pipeline_name, state...","[BIGINT, BIGINT, VARCHAR, VARCHAR, TIMESTAMP W...",False
2,sakila,staging,_dlt_version,"[version, engine_version, inserted_at, schema_...","[BIGINT, BIGINT, TIMESTAMP WITH TIME ZONE, VAR...",False
3,sakila,staging,actor,"[actor_id, first_name, last_name, last_update,...","[DECIMAL(38,9), VARCHAR, VARCHAR, TIMESTAMP, V...",False
4,sakila,staging,address,"[address_id, address, address2, district, city...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, BIGINT, VA...",False
5,sakila,staging,category,"[category_id, name, last_update, _dlt_load_id,...","[BIGINT, VARCHAR, TIMESTAMP, VARCHAR, VARCHAR]",False
6,sakila,staging,city,"[city_id, city, country_id, last_update, _dlt_...","[BIGINT, VARCHAR, BIGINT, TIMESTAMP, VARCHAR, ...",False
7,sakila,staging,country,"[country_id, country, last_update, _dlt_load_i...","[BIGINT, VARCHAR, TIMESTAMP, VARCHAR, VARCHAR]",False
8,sakila,staging,customer,"[customer_id, store_id, first_name, last_name,...","[BIGINT, BIGINT, VARCHAR, VARCHAR, VARCHAR, BI...",False
9,sakila,staging,film,"[film_id, title, description, release_year, la...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, BIGINT, BI...",False


### Load and Register Tables from Duckdb Staging

In [9]:
la = {}
with duckdb.connect("data/sakila.duckdb") as conn:
    for name in desc["name"]:
        la[name] = conn.sql(f"FROM staging.{name};").df()
        duckdb.register(name, la[name])
        


### a) Which movies are longer than 3 hours (180 minutes), show the title and its length?

In [11]:
la['film'].head(3)

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update,_dlt_load_id,_dlt_id
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,<NA>,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2021-03-06 15:52:00,1765282237.85467,8UeHo0d4NsKrhQ
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,<NA>,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2021-03-06 15:52:00,1765282237.85467,U985ypLic88u7A
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,<NA>,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2021-03-06 15:52:00,1765282237.85467,qxu5GG1U0R7RUQ


In [14]:
duckdb.sql("SELECT title, length FROM film WHERE length > 180 ORDER BY length;").df()

,title,length
0,LAWLESS VISION,181
1,WILD APOLLO,181
2,STAR OPERATION,181
3,RUNAWAY TENENBAUMS,181
4,ANALYZE HOOSIERS,181
5,HAUNTING PIANIST,181
6,LOVE SUICIDES,181
7,HOTEL HAPPINESS,181
8,INTRIGUE WORST,181
9,JACKET FRISCO,181


### b) Which movies have the word "love" in its title? Show the following columns title, rating, length, description

In [20]:
duckdb.sql("SELECT title, rating, length, description FROM film WHERE title ILIKE '%love%' ORDER BY length DESC;").df()

,title,rating,length,description
0,LOVE SUICIDES,R,181,A Brilliant Panorama of a Hunter And a Explore...
1,LAWRENCE LOVE,NC-17,175,A Fanciful Yarn of a Database Administrator An...
2,IDAHO LOVE,PG-13,172,A Fast-Paced Drama of a Student And a Crocodil...
3,LOVERBOY ATTACKS,PG-13,162,A Boring Story of a Car And a Butler who must ...
4,INDIAN LOVE,NC-17,135,A Insightful Saga of a Mad Scientist And a Mad...
5,IDENTITY LOVER,PG-13,119,A Boring Tale of a Composer And a Mad Cow who ...
6,GRAFFITI LOVE,PG,117,A Unbelieveable Epistle of a Sumo Wrestler And...
7,STRANGELOVE DESIRE,NC-17,103,A Awe-Inspiring Panorama of a Lumberjack And a...
8,LOVER TRUMAN,G,75,A Emotional Yarn of a Robot And a Boy who must...
9,LOVELY JINGLE,PG,65,A Fanciful Yarn of a Crocodile And a Forensic ...


### c) Calculate descriptive statistics on the length column, The Manager wants, shortest, average, median and longest movie length

In [24]:
duckdb.sql("SELECT MIN(length) AS min_length_minutes, ROUND(AVG(length)) AS avg_length_minutes, MEDIAN(length) AS mean_length_minutes, MAX(length) AS max_length_minutes FROM film; ").df()

,min_length_minutes,avg_length_minutes,mean_length_minutes,max_length_minutes
0,46,115.0,114.0,185


 ### d) The rental rate is the cost to rent a movie and the rental duration is the number of days a customer can keep the movie.
 ### The Manager wants to know the 10 most expensive movies to rent per day.

In [25]:
la["film"].head(1)

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update,_dlt_load_id,_dlt_id
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,<NA>,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2021-03-06 15:52:00,1765282237.85467,8UeHo0d4NsKrhQ


In [30]:
duckdb.sql("SELECT title, rental_rate/rental_duration AS rate_per_day FROM film ORDER BY rate_per_day DESC LIMIT 10;").df()

,title,rate_per_day
0,AMERICAN CIRCUS,1.663333
1,BACKLASH UNDEFEATED,1.663333
2,BILKO ANONYMOUS,1.663333
3,BEAST HUNCHBACK,1.663333
4,CARIBBEAN LIBERTY,1.663333
5,AUTUMN CROW,1.663333
6,CASPER DRAGONFLY,1.663333
7,ACE GOLDFINGER,1.663333
8,BEHAVIOR RUNAWAY,1.663333
9,CASUALTIES ENCINO,1.663333


### e) Which actors have played in most movies? Show the top 10 actors with the number of movies they have played in.

In [35]:
duckdb.sql("SELECT  a.first_name || ' ' || a.last_name AS actor_name, COUNT( distinct fa.film_id) AS total_films FROM actor a LEFT JOIN film_actor fa ON a.actor_id = fa.actor_id GROUP BY  actor_name  ORDER BY total_films DESC LIMIT 10;").df()

,actor_name,total_films
0,SUSAN DAVIS,54
1,GINA DEGENERES,42
2,WALTER TORN,41
3,MARY KEITEL,40
4,MATTHEW CARREY,39
5,SANDRA KILMER,37
6,SCARLETT DAMON,36
7,ANGELA WITHERSPOON,35
8,HENRY BERRY,35
9,GROUCHO DUNST,35
